In [ ]:
import os

In [ ]:
jvm = spark.sparkContext._jvm
gdb = jvm.com.esri.gdb.FileGDB

In [ ]:
gdb_path = os.path.join("data","Miami.gdb")
for tab in gdb.listTables(gdb_path):
    print(tab)

In [ ]:
df = spark.read \
    .format('com.esri.gdb') \
    .options(path=gdb_path, name='Broadcast') \
    .load()

In [ ]:
df.printSchema()

In [ ]:
df.count()

In [ ]:
df.show()

In [ ]:
df.registerTempTable("bc")

In [ ]:
sql("""
select mmsi,dt1,x1,y1,dt2,x2,y2 from (
select MMSI mmsi,
BaseDateTime as dt1,
Shape.x as x1,
Shape.y as y1,
lead(BaseDateTime) over (partition by MMSI order by BaseDateTime) as dt2,
lead(Shape.x) over (partition by MMSI order by BaseDateTime) as x2,
lead(Shape.y) over (partition by MMSI order by BaseDateTime) as y2
from bc
)
where dt1 < dt2
""").show(100, truncate=True)